In [ ]:
import os
import gc
from tqdm import tqdm
import illustris_python as il
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits import mplot3d
import math
import h5py


# %matplotlib widget
basePath = "./sims.TNG/TNG100-1/output"

initial_snapshot = 33
end_snapshot = 99

halos_fields = ["GroupFirstSub", "GroupNsubs", "GroupMass", "GroupPos"]
subhalos_fields = ["SubhaloGrNr", "SubhaloMass", "SubhaloPos"]
tree_fields = ["SubfindID", "SnapNum"]

h = 0.6774  # km/Mpc

m_dm = 4.5e-5  # E10M_Sum
m_dm_h = m_dm * h

Critial_Density = 1.27e-8  # E10M_Sum/kpc^3
Critial_Density_h = Critial_Density / h**2  # E10M_Sum/(kpc/h)^3


def Detect_Mkdir(Path):
    if not (os.path.exists(Path)):
        os.makedirs(Path)
    return Path


FilePath = Detect_Mkdir(os.path.join('./TNG100_Final_File'))
Final_File_List = ['Halo_Array', 'Host_Array', 'MMS_Array', 'First_App_Snap_Array', 'First_App_Distance_Array',
                   'Snap99_Distance_Array', 'Snap99_Radial_Velocity_Array', 'Snap99_Virial_Radius_Array', 'Is_First_Approach', 'Is_First_Infall']
for filename in Final_File_List:
    if os.path.exists(os.path.join(FilePath, filename)+'.npy'):
        globals()[filename] = np.load(
            (os.path.join(FilePath, filename)+'.npy'))


In [ ]:
def Calculate_Halo_Dict(halo_index, host_index, mms_index):
    host_tree = il.sublink.loadTree(
        basePath, end_snapshot, host_index, fields=tree_fields, onlyMPB=1
    )
    mms_tree = il.sublink.loadTree(
        basePath, end_snapshot, mms_index, fields=tree_fields, onlyMPB=1
    )

    satellite_velocity_list = list()
    host_mms_distance = list()
    host_position_list = list()
    mms_position_list = list()
    for current_snapshot in range(initial_snapshot, end_snapshot + 1):
        header = il.groupcat.loadHeader(basePath, current_snapshot)
        scale_factor = header["Time"]
        #age = Age_Array[np.where(Snapshot_Array == current_snapshot)[0][0]]

        host_subhalo_index = host_tree["SubfindID"][
            np.where(host_tree["SnapNum"] == current_snapshot)[0]
        ]
        mms_subhalo_index = mms_tree["SubfindID"][
            np.where(mms_tree["SnapNum"] == current_snapshot)[0]
        ]

        if len(mms_subhalo_index) == 0 or len(host_subhalo_index) == 0:
            satellite_velocity_list.append(np.nan)
            host_mms_distance.append(np.nan)
            host_position_list.append(np.array([np.nan, np.nan, np.nan]))
            mms_position_list.append(np.array([np.nan, np.nan, np.nan]))
            continue

        host_subhalo_dict = il.groupcat.loadSingle(
            basePath, current_snapshot, -1, host_subhalo_index
        )
        mms_subhalo_dict = il.groupcat.loadSingle(
            basePath, current_snapshot, -1, mms_subhalo_index
        )

        current_host_comoving_position = host_subhalo_dict["SubhaloPos"]
        current_mms_comoving_position = mms_subhalo_dict["SubhaloPos"]

        current_host_physical_position = current_host_comoving_position * scale_factor
        current_mms_physical_position = current_mms_comoving_position * scale_factor

        current_host_physical_position = current_host_physical_position % 35000
        current_mms_physical_position = current_mms_physical_position % 35000

        current_difference_physical_coordinates = (
            current_mms_physical_position - current_host_physical_position
        )
        distance = np.linalg.norm(current_difference_physical_coordinates)

        if len(host_mms_distance):
            if np.abs(distance-host_mms_distance[-1])>60:
                satellite_velocity_list.append(np.nan)
                host_mms_distance.append(np.nan)
                host_position_list.append(np.array([np.nan, np.nan, np.nan]))
                mms_position_list.append(np.array([np.nan, np.nan, np.nan]))
                continue

        host_velocity = host_subhalo_dict["SubhaloVel"]
        mms_velocity = mms_subhalo_dict["SubhaloVel"]

        Vel_diff = mms_velocity - host_velocity
        Vel_Radius = (
            np.dot(Vel_diff, current_difference_physical_coordinates) / distance
        )

        satellite_velocity_list.append(Vel_Radius)
        host_mms_distance.append(distance)

        host_position_list.append(current_host_physical_position)
        mms_position_list.append(current_mms_physical_position)

        gc.collect()

    del host_tree,mms_tree,header,scale_factor
    del host_subhalo_dict,host_subhalo_index
    del mms_subhalo_dict,mms_subhalo_index
    del current_host_comoving_position,current_mms_comoving_position
    del current_host_physical_position,current_mms_physical_position
    del distance

    satellite_radius_velocity = np.array(satellite_velocity_list)
    host_mms_distance = np.array(host_mms_distance)
    host_position_array = np.array(host_position_list)
    mms_position_array = np.array(mms_position_list)

    del satellite_velocity_list,host_position_list,mms_position_list

    return_data = dict()

    return_data["Host_Index"] = host_index
    return_data["MMS_Index"] = mms_index
    return_data["Halo_Index"] = halo_index

    return_data["RadialVelocity"] = satellite_radius_velocity
    return_data["Distance"] = host_mms_distance
    return_data["HostPosition"] = host_position_array
    return_data["MMSPosition"] = mms_position_array
    return_data["Snapshots"] = np.arange(initial_snapshot, end_snapshot + 1)

    del satellite_radius_velocity,host_mms_distance,host_position_array,mms_position_array
    gc.collect()

    return return_data


In [4]:
hdf5_File_Path='./APMS_TNG100_hdf5'

start_sub_index = len(os.listdir(Detect_Mkdir(hdf5_File_Path)))-1
step_length = Halo_Array.shape[0]+1
end = start_sub_index+step_length

if end > Halo_Array.shape[0]:
    end = Halo_Array.shape[0]

if start_sub_index<0: start_sub_index=0

for Subindex in tqdm(range(start_sub_index, end)):
    halo_index = Halo_Array[Subindex]
    host_index = Host_Array[Subindex]
    MMS_index = MMS_Array[Subindex]

    halo_dict = Calculate_Halo_Dict(halo_index, host_index, MMS_index)

    with h5py.File((os.path.join(hdf5_File_Path, 'Halo-'+str(halo_dict['Halo_Index']))+'.hdf5'), 'w') as f:
        for key, value in halo_dict.items():
            f.create_dataset(key, data=value)


100%|██████████| 4396/4396 [16:21:19<00:00, 13.39s/it]  


In [ ]:
hdf5_files = [f for f in os.listdir(hdf5_File_Path) if f.endswith('.hdf5')]
for file_name in hdf5_files:
    with h5py.File(os.path.join(hdf5_File_Path, file_name), 'r') as f:
            for key in f.keys():
                globals()[key] = f[key][()]




In [5]:
len(os.listdir(hdf5_File_Path))


4396

In [6]:
os.path.getsize(hdf5_File_Path)


262144